In [14]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import csv
import matplotlib.pyplot as plt

# keras imports
import keras
from keras import backend as K
backend_keras = keras.backend.backend()
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
print("keras is using", backend_keras,"as the backend")

# import a whole bunch of other packeges
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import keras
import pickle
import cv2
import os

keras is using tensorflow as the backend


In [15]:
def show_image(arr):
    two_d = (np.reshape(arr, (28,28))*255).astype(np.uint8)
    plt.imshow(two_d, cmap='Greys', interpolation='nearest')
    plt.show()

# load the PADDED TRAIN IMAGES
train_images = np.load("../train_images.npy")


Transform and normalize images??

In [16]:
# load the training labels into a dictionary,
# also load it into a list, haven't decided which one is better yet
labels_dic = {}
labels = []
with open('../train_labels.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        try: 
            labels_dic[int(row[0])] = int(row[1])
            labels.append(int(row[1]))
        except: print(row)
            
len(labels_dic)

['ID', 'label']


50000

In [17]:
# normalize images
train_images = np.array(train_images, dtype="float") / 255.0

## reshape the images so that they have 3 channels, 3 identical copies
#train_images_3 = [[[[k,k,k] for k in j] for j in i] for i in train_images]
#train_images_3 = np.array(train_images_3)    

labels = np.array(labels)
train_labels = []
for i in labels:

    label = np.zeros(10)
    label[i]=1
    train_labels.append(label)

train_labels = np.array(train_labels)
#print(labels[2])

In [18]:
print(train_images.shape)
print(train_images_3.shape)

(50000, 28, 28)


NameError: name 'train_images_3' is not defined

In [19]:
print(train_labels.shape,train_images.shape)

(50000, 10) (50000, 28, 28)


## Loading Keras Model

following this guid
https://www.learnopencv.com/keras-tutorial-using-pre-trained-imagenet-models/

and also following this guid
https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

In [16]:
# load the VGG model
vgg_conv = vgg16.VGG16(include_top=False,
                        weights='imagenet',
                        input_shape=(32,32,3))

# load the inception_V3 model
inception_model = inception_v3.InceptionV3(weights='imagenet')

# load the resnet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')

# load the mobilenet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

### Freeze the required layers

In [169]:
print("vgg_conv layers")
for layer in vgg_conv.layers:
    print(layer)
    
print("\n\ninception_model layers")
for layer in inception_model.layers:
    print(layer)
    
print("\n\nresnet_model layers")
for layer in resnet_model.layers:
    print(layer)
    
print("\n\nmobilenet_model layers")
for layer in mobilenet_model.layers:
    print(layer)

vgg_conv layers


inception_model layers


resnet_model layers


mobilenet_model layers


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, input_shape=(28,28,1)), padding='same')
model.add(Conv2D(filters=32, kernel_size=3), padding='same')
model.add(MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None))
model.add(Dense(350, activation='relu', input_dim=784))
model.add(Dense(220, activation='relu'))

model.add(Dense(120, activation='relu'))
model.add(Dense(10, activation='softmax'))



In [20]:
# compile my model
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

#model.compile(optimizer='rmsprop',
 #             loss='categorical_crossentropy',
  #            metrics=['accuracy'])

### Create a new model

In [21]:
# show a summary of the model. Check the number of trainable params
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 350)               274750    
_________________________________________________________________
dense_10 (Dense)             (None, 220)               77220     
_________________________________________________________________
dense_11 (Dense)             (None, 120)               26520     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1210      
Total params: 379,700
Trainable params: 379,700
Non-trainable params: 0
_________________________________________________________________


### batchsize

In [21]:
# change batchsize according to ram
train_batchsize = 64
val_batchsize=32

### augmentation

In [23]:
print(tim.shape)
print(labels_aug.shape)

NameError: name 'tim' is not defined

In [24]:
# rotate 
train_images
train_rot_90 = np.array([np.rot90(i,k=1) for i in train_images])
train_rot_180 = np.array([np.rot90(i,k=2) for i in train_images])
train_rot_270 = np.array([np.rot90(i,k=3) for i in train_images])
ref = np.array([np.transpose(i) for i in train_images])
ref_90 = np.array([np.rot90(i,k=1) for i in train_images])
ref_180 = np.array([np.rot90(i,k=2) for i in train_images])
ref_270 = np.array([np.rot90(i,k=3) for i in train_images])


tim = np.concatenate((train_images,
                     train_rot_90,
                     train_rot_180,
                     train_rot_270,
                     ref,
                     ref_90,
                     ref_180,
                     ref_270))

labels_aug = np.concatenate((train_labels,train_labels,
                            train_labels,train_labels,
                            train_labels,train_labels,
                            train_labels,train_labels))

labels=labels_aug

In [25]:
train_images = tim[:]
print(train_images.shape)
print(labels_aug.shape)

(400000, 28, 28)
(400000, 10)


#### train val split

In [28]:
# flatten first
print(train_images.shape)
train_images_flatten = np.array([np.ndarray.flatten(i) for i in train_images])
print(train_images_flatten.shape)

# train val split
r = 0.8
l = len(train_images_flatten)
cutoff = int(r*l)
x_train = train_images_flatten[:cutoff]
y_train = train_labels[:cutoff]
x_val = train_images_flatten[cutoff:]
y_val = train_labels[cutoff:]

(400000, 28, 28)
(400000, 784)


### train the model

In [29]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [30]:
# train the model
model.fit(train_images_flatten,
                    labels_aug,
                    epochs=25,
                    batch_size=32,
                    shuffle=True,
                    validation_split=0.2)

Train on 320000 samples, validate on 80000 samples
Epoch 1/25
320000/320000 [==============================] - 75s 236us/step - loss: 0.4160 - accuracy: 0.8449 - val_loss: 0.4433 - val_accuracy: 0.8337
Epoch 2/25
320000/320000 [==============================] - 68s 212us/step - loss: 0.4132 - accuracy: 0.8457 - val_loss: 0.4418 - val_accuracy: 0.8331
Epoch 3/25
320000/320000 [==============================] - 69s 215us/step - loss: 0.4099 - accuracy: 0.8466 - val_loss: 0.4425 - val_accuracy: 0.8335
Epoch 4/25
320000/320000 [==============================] - 71s 222us/step - loss: 0.4080 - accuracy: 0.8474 - val_loss: 0.4297 - val_accuracy: 0.8372
Epoch 5/25
320000/320000 [==============================] - 65s 203us/step - loss: 0.4070 - accuracy: 0.8483 - val_loss: 0.4658 - val_accuracy: 0.8271
Epoch 6/25
320000/320000 [==============================] - 67s 210us/step - loss: 0.4067 - accuracy: 0.8479 - val_loss: 0.4173 - val_accuracy: 0.8438
Epoch 7/25
320000/320000 [=================

### save the model

In [31]:
#train_images_flatten[0]

In [5]:
# save the model
model_name = 'percent08_opt_3_hidd_model'
#model.save(model_name+'.h5')

### save the model to json
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [6]:
from keras.models import model_from_json
import numpy
import os

In [34]:
# serialize model to JSON
model_json = model.to_json()
with open(model_name+".json","w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(model_name+".h5")
print("Saved model to disk")


Saved model to disk


### Load model from json

In [11]:
# load json and create model
json_file = open(model_name+'.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_name+".h5")
print("loaded model from disk")

loaded model from disk


In [10]:
import pandas as pd

x_test = np.load('../test_images.npy').squeeze()
x_test = x_test.reshape(len(x_test), -1)

# normalize
x_test = np.array(x_test, dtype="float") / 255.0


y_test = model.predict(x_test)
y_test = np.array([i.argmax() for i in y_test])


df_test = pd.read_csv('../sample_submission.csv')
df_test['label'] = y_test
df_test.to_csv('submission.csv', index=False)

In [12]:
model = loaded_model